In [1]:
# we wiil go through data --> model --> train --> test --> confusion matrix

# libraries we will want 
from tensorflow.keras.applications import * 
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
from tensorflow.keras import optimizers
import tensorflow as tf
import matplotlib.pyplot as plt

# use gpu instead of cpu 
my_gpu = tf.config.experimental.list_physical_devices('GPU')[0]

# avoid oom error 
tf.config.experimental.set_memory_growth(my_gpu,True)

In [3]:
import os

train_dir = os.path.join("train") 

test_dir = os.path.join("test") 

val_dir = os.path.join("validation") 

train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (256, 256), batch_size = 32)

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (256, 256), batch_size = 32)

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(val_dir, target_size = (256, 256), batch_size = 32)

Found 83436 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.
Found 24 images belonging to 4 classes.


In [4]:
vgg_16 = tf.keras.applications.VGG16(
    include_top = False, 
    weights = 'imagenet', 
    input_shape = (256,256,3), 
)

In [5]:
vgg_16.trainable = False

model_vgg = tf.keras.models.Sequential([
    vgg_16,
    tf.keras.layers.Conv2D(128, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),# modifying final layers of VGG-16
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Dense(4, activation = 'softmax')
])

In [6]:
metrics = ['accuracy',
                tf.keras.metrics.AUC(),
                tf.keras.metrics.Precision(), 
                tf.keras.metrics.Recall()]

In [7]:
model_vgg.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = metrics)
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 128)         589952    
                                                                 
 p_re_lu (PReLU)             (None, 8, 8, 128)         8192      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          73792     
                                                                 
 p_re_lu_1 (PReLU)           (None, 8, 8, 64)          4096      
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 100)               4

In [8]:
# train : 
logdir='logs'

# we do something like check point for us to return back to a specific location 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# train 
hist = model_vgg.fit(train_generator, epochs=30, validation_data=validation_generator, callbacks=[tensorboard_callback])

In [9]:
model_vgg.save(os.path.join("models","model_vgg16_unbalanced.h5"))